In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Util functions for fetching data
from modules.fetcher import Fetcher

def fetch_data(exchange, market):
    fetcher = Fetcher()
    funding_df = fetcher.fetch_funding_rate_history_until_start(exchange, market)
    funding_df['datetime'] = funding_df['datetime'].dt.tz_localize(None)

    start_funding_time = funding_df['timestamp'].min()
    end_funding_time = funding_df['timestamp'].max()

    price_df = fetcher.fetch_ohlc(exchange, market, start_funding_time, end_funding_time)
    price_df['datetime'] = price_df['datetime'].dt.tz_localize(None)

    start_price_time = price_df['timestamp'].min()
    end_price_time = price_df['timestamp'].max()

    min_time = max(start_funding_time, start_price_time)
    max_time = min(end_funding_time, end_price_time)

    min_datetime = pd.to_datetime(min_time, unit='s')
    max_datetime = pd.to_datetime(max_time, unit='s')

    funding_df = funding_df[(funding_df['datetime'] >= min_datetime) & (funding_df['datetime'] <= max_datetime)]
    price_df = price_df[(price_df['datetime'] >= min_datetime) & (price_df['datetime'] <= max_datetime)]

    result_df = pd.merge_asof(funding_df, price_df, on=['datetime'], tolerance=pd.Timedelta('1h'), direction='nearest')
    result_df['open'] = result_df['open'].fillna(method='ffill')
    result_df['high'] = result_df['high'].fillna(method='ffill')
    result_df['low'] = result_df['low'].fillna(method='ffill')
    result_df['close'] = result_df['close'].fillna(method='ffill')
    result_df['timestamp'] = result_df['timestamp_x']

    result_df['datetime'] = result_df['datetime'].apply(lambda x: pd.to_datetime(x))

    return result_df[['datetime', 'timestamp', 'open', 'high', 'low', 'close', 'funding_rate']]

In [ ]:
# Util functions for calculating pnl
def get_backtest_result(input_df, l, fee = 0.001, penalty = 0.01, maintenance_margin = 0.05, stop_loss_margin = 0.0625):
    df = input_df.copy()
    for index, row in enumerate(df.iterrows()):
        if index == 0:
            df['clt'] = 1
            df['leverage'] = l
            df['entry'] = 0
            df['pos_size'] = 0
            df['change'] = 0
            df['change_pnl'] = 0
            df['funding'] = 0
            df['funding_pnl'] = 0
            df['margin'] = 0

            df['mm'] = df['clt'] * df['leverage'] * maintenance_margin
            df['mm_sl'] = df['clt'] * df['leverage'] * stop_loss_margin

            df['is_liq'] = False
            df['is_sl'] = False

            df['fee'] = -fee * df['leverage']
            df['penalty'] = 0
            df['final_pnl'] = 0
        else:
            prev_df = df.loc[index - 1]
            # check if is there was a trade in the previous record
            traded = prev_df['fee'] != 0

            # calculate new clt from previous clt + fee + penalty + funding pnl (if traded)
            new_clt = prev_df['clt'] + prev_df['fee'] + prev_df['penalty'] + (prev_df['funding_pnl'] if traded else 0)

            if new_clt == 0:
                df.loc[index, 'clt'] = 0
                df.loc[index, 'entry'] = 0
                df.loc[index, 'pos_size'] = 0
                df.loc[index, 'change'] = 0
                df.loc[index, 'change_pnl'] = 0
                df.loc[index, 'funding'] = 0
                df.loc[index, 'funding_pnl'] = 0
                df.loc[index, 'margin'] = 0
                df.loc[index, 'mm'] = 0
                df.loc[index, 'mm_sl'] = 0
                df.loc[index, 'is_liq'] = False
                df.loc[index, 'is_sl'] = False
                df.loc[index, 'fee'] = 0
                df.loc[index, 'penalty'] = 0
                df.loc[index, 'final_pnl'] = -1
            else:
                price = float(df.loc[index, 'close'])
                funding_rate = float(df.loc[index, 'funding_rate'])

                df.loc[index, 'clt'] = max(new_clt, 0)
                # Entry price of the current position
                df.loc[index, 'entry'] = price if traded else prev_df['entry']
                # Size of the current position
                df.loc[index, 'pos_size'] = price * df.loc[index, 'clt'] * df.loc[index, 'leverage']
                # Change of the current position (compared to entry price)
                df.loc[index, 'change'] = (price - df.loc[index, 'entry']) / df.loc[index, 'entry'] if df.loc[index, 'entry'] != 0 else 0
                # Change pnl (we treat any changes as loss since we will either close the short position or long position if the price hits the stop loss)
                df.loc[index, 'change_pnl'] = -abs(df.loc[index, 'change'] * df.loc[index, 'leverage'])
                # Funding payment comes from the funding rate and the position size (collateral + change pnl)
                df.loc[index, 'funding'] = (df.loc[index, 'clt'] - df.loc[index, 'change'] / 2) * funding_rate * df.loc[index, 'leverage'] / 2
                # Funding pnl is accumulated while there is no trading. If there is a trade, we reset the funding pnl and set it to the current funding payment
                df.loc[index, 'funding_pnl'] = df.loc[index, 'funding'] if traded else df.loc[index, 'funding'] + df.loc[index - 1, 'funding_pnl']
                # Calculate current margin to check stop loss and liquidation
                df.loc[index, 'margin'] = (df.loc[index, 'clt'] + df.loc[index, 'change_pnl'] + df.loc[index, 'funding_pnl']) / df.loc[index, 'clt'] if df.loc[index, 'clt'] != 0 else 0

                # Calculate maintenance margin for liquidation
                df.loc[index, 'mm'] = df.loc[index, 'clt'] * df.loc[index, 'leverage'] * maintenance_margin
                # Calculate maintenance margin for stop loss
                df.loc[index, 'mm_sl'] = df.loc[index, 'clt'] * df.loc[index, 'leverage'] * stop_loss_margin

                # Check if the current position is liquidated
                df.loc[index, 'is_liq'] = df.loc[index, 'margin'] < df.loc[index,'mm']
                # Check if the current position is stop loss
                df.loc[index, 'is_sl'] = df.loc[index, 'margin'] < df.loc[index, 'mm_sl']

                # Include fee if liquidation or stop loss occur
                df.loc[index, 'fee'] = -fee * df.loc[index, 'leverage'] if df.loc[index, 'is_liq'] or df.loc[index, 'is_sl'] else 0
                # Include penalty if liquidation occur
                df.loc[index, 'penalty'] = -fee * df.loc[index, 'leverage'] if df.loc[index, 'is_liq'] else 0
                # Calculate final pnl
                df.loc[index, 'final_pnl'] = df.loc[index, 'clt'] - 1 + df.loc[index, 'funding_pnl']
    return df

In [ ]:
# Util functions for hodl pnl
def get_hodl_result(input_df):
    df = input_df.copy()
    df = df.sort_values(by='datetime', ascending=True)
    df = df.reset_index(drop=True)
    df['close'] = df['close'].astype(float)
    first_price = df.loc[0, 'close']
    df['pnl'] = (df['close'] - first_price) / first_price
    return df

In [ ]:
# Util functions for max drawdown pnl
def max_drawdown(values):
    # Calculate the running maximum
    running_max = values.expanding(min_periods=1).max()
    # Calculate the drawdown
    drawdowns = running_max - values
    drawdowns.replace(float('inf'), 0, inplace=True)

    return drawdowns.max()

In [ ]:
# Fetch and save funding rate and price history

# Input
exchanges_markets = [
    {
        "binance": "BTCUSDT",
        "bitmex": "XBTUSD",
        "apollox": "BTCUSDT",
        "drift": "BTC-PERP"
    },
    {
        "binance": "ETHUSDT",
        "bitmex": "ETHUSD",
        "apollox": "ETHUSDT",
        "drift": "ETH-PERP"
    },
    {
        "binance": "SOLUSDT",
        "bitmex": "SOLUSD",
        "apollox": "SOLUSDT",
        "drift": "SOL-PERP"
    },
    {
        "binance": "XRPUSDT",
        "bitmex": "XRPUSD",
        "apollox": "XRPUSDT",
        "drift": "XRP-PERP"
    },
    {
        "binance": "BNBUSDT",
        "bitmex": "BNBUSD",
        "apollox": "BNBUSDT",
        "drift": "BNB-PERP"
    },
]

# for exchanges_market in exchanges_markets:
#     for exchange, market in exchanges_market.items():
#          data = fetch_data(exchange, market)
#          data.to_csv(f'./data/{exchange}_{market}.csv', index=False)

In [ ]:
# Compute funding rate PNL with only one market
exchanges_market = {
    "binance": "BTCUSDT",
    "bitmex": "XBTUSD",
    "apollox": "BTCUSDT",
    "drift": "BTC-PERP",
}
benchmark_exchange = "binance"

# Initial variables
min_time = 0
max_time = datetime.now().timestamp()

data_df_map = {}
results = {}

leverage = 1

# Read data from file
for exchange, market in exchanges_market.items():
    data_df = pd.read_csv(f"./data/{exchange}_{market}.csv")
    data_df_map[exchange] = data_df

# calculate min max time
for exchange, market in exchanges_market.items():
    data_df = data_df_map[exchange]
    min_time = max(min_time, data_df["timestamp"].min())
    max_time = min(max_time, data_df["timestamp"].max())

# Calculate funding rate backtest result
for exchange, market in exchanges_market.items():
    data_df = data_df_map[exchange]
    data_df['datetime'] = data_df['datetime'].apply(lambda x: pd.to_datetime(x))

    data_df = data_df[
        (data_df["timestamp"] >= min_time) & (data_df["timestamp"] <= max_time)
    ]

    data_df = data_df.sort_values(by='datetime', ascending=True)
    data_df = data_df.reset_index(drop=True)

    result_df = get_backtest_result(data_df, leverage)
    results[exchange] = result_df

hodl_df = get_hodl_result(results[benchmark_exchange])

# Results are store in 'results' and 'hodl_df' variables

In [ ]:
# Plot funding rate arb PNL with HODL PNL
exchanges_market = {
    "binance": "BTCUSDT",
    "bitmex": "XBTUSD",
    "drift": "BTC-PERP"
}
line_styles = {"binance": "--", "bitmex": ":", "apollox": "--", "drift": "-."}
dash_styles = {"binance": [8, 4], "bitmex": [1, 3], "apollox": [8, 4], "drift":[1, 2, 10, 2]}
hodl_line_style = "-"
hodl_dash_style = []

fig, ax = plt.subplots(figsize=(20, 12))

# Plot funding arb result
for exchange, market in exchanges_market.items():
    result_df = results[exchange]
    line,  = ax.plot(
        result_df["datetime"],
        result_df["final_pnl"] * 100,
        label=f"{exchange} {market}",
        linestyle=line_styles[exchange],
        color="black",
    )
    line.set_dashes(dash_styles[exchange])

# Plot hodl result
ax.plot(hodl_df['datetime'], hodl_df['pnl'] * 100, label=f'HODL binance BTCUSDT', linestyle=hodl_line_style, dashes=hodl_dash_style, color='black')

# Chart settings
lines, labels = ax.get_legend_handles_labels()
plt.xlabel("Date", fontsize=24)
plt.ylabel("PnL (%)", fontsize=24)
ax.legend(lines, labels, loc="upper right", bbox_to_anchor=(0.3, 0.95), fontsize=20)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

# Ensure a tight layout to avoid label/legend clipping
plt.tight_layout()

plt.savefig('./images/basic_funding_hodl_pnl.png', format='png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Plot funding rate arb PNL only
exchanges_market = {
    "binance": "BTCUSDT",
    "bitmex": "XBTUSD",
    "apollox": "BTCUSDT",
    "drift": "BTC-PERP"
}
line_styles = {"binance": "-", "bitmex": ":", "apollox": "--", "drift": "-."}
dash_styles = {"binance": [], "bitmex": [1, 3], "apollox": [8, 4], "drift":[1, 2, 10, 2]}

fig, ax = plt.subplots(figsize=(20, 12))

# Plot funding arb result
for exchange, market in exchanges_market.items():
    result_df = results[exchange]
    line,  = ax.plot(
        result_df["datetime"],
        result_df["final_pnl"] * 100,
        label=f"{exchange} {market}",
        linestyle=line_styles[exchange],
        color="black",
    )
    line.set_dashes(dash_styles[exchange])

# Chart settings
lines, labels = ax.get_legend_handles_labels()
plt.xlabel("Date", fontsize=24)
plt.ylabel("PnL (%)", fontsize=24)
ax.legend(lines, labels, loc="upper right", bbox_to_anchor=(0.3, 0.95), fontsize=20)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

# Ensure a tight layout to avoid label/legend clipping
plt.tight_layout()

plt.savefig('./images/basic_funding_pnl.png', format='png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Compute funding rate PNL with only one market

# Input
exchanges_markets = [
    {
        "binance": "BTCUSDT",
        "bitmex": "XBTUSD",
        "apollox": "BTCUSDT",
        "drift": "BTC-PERP"
    },
    {
        "binance": "ETHUSDT",
        "bitmex": "ETHUSD",
        "apollox": "ETHUSDT",
        "drift": "ETH-PERP"
    },
    {
        "binance": "SOLUSDT",
        "bitmex": "SOLUSD",
        "apollox": "SOLUSDT",
        "drift": "SOL-PERP"
    },
    {
        "binance": "XRPUSDT",
        "bitmex": "XRPUSD",
        "apollox": "XRPUSDT",
        "drift": "XRP-PERP"
    },
    {
        "binance": "BNBUSDT",
        "bitmex": "BNBUSD",
        "apollox": "BNBUSDT",
        "drift": "BNB-PERP"
    },
]

benchmark_exchange = "binance"

# Initial variables
min_time = 0
max_time = datetime.now().timestamp()

data_df_map = {}
hodl_df_map = {}
results = {}

leverage = 1

for exchanges_market in exchanges_markets:
    min_time = 0
    max_time = datetime.now().timestamp()

    # Read data from file
    for exchange, market in exchanges_market.items():
        data_df = pd.read_csv(f"./data/{exchange}_{market}.csv")
        if (market not in data_df_map):
            data_df_map[market] = {}
        data_df_map[market][exchange] = data_df

    # calculate min max time
    for exchange, market in exchanges_market.items():
        data_df = data_df_map[market][exchange]
        min_time = max(min_time, data_df["timestamp"].min())
        max_time = min(max_time, data_df["timestamp"].max())

    # Fetch all funding rates and check date range
    for exchange, market in exchanges_market.items():
        data_df = data_df_map[market][exchange]
        data_df['datetime'] = data_df['datetime'].apply(lambda x: pd.to_datetime(x))

        data_df = data_df[
            (data_df["timestamp"] >= min_time) & (data_df["timestamp"] <= max_time)
        ]

        data_df = data_df.sort_values(by='datetime', ascending=True)
        data_df = data_df.reset_index(drop=True)

        result_df = get_backtest_result(data_df, leverage)
        if (market not in results):
            results[market] = {}
        results[market][exchange] = result_df

    benchmark_market = exchanges_market[benchmark_exchange]
    hodl_df_map[benchmark_market] = get_hodl_result(results[benchmark_market][benchmark_exchange])

# Results are store in 'results' and 'hodl_df_map' variables

In [ ]:
# Summary table

# Inputs
exchanges_markets = [
    {
        "binance": "BTCUSDT",
        "bitmex": "XBTUSD",
        "apollox": "BTCUSDT",
        "drift": "BTC-PERP"
    },
    {
        "binance": "ETHUSDT",
        "bitmex": "ETHUSD",
        "apollox": "ETHUSDT",
        "drift": "ETH-PERP"
    },
    {
        "binance": "SOLUSDT",
        "bitmex": "SOLUSD",
        "apollox": "SOLUSDT",
        "drift": "SOL-PERP"
    },
    {
        "binance": "XRPUSDT",
        "bitmex": "XRPUSD",
        "apollox": "XRPUSDT",
        "drift": "XRP-PERP"
    },
    {
        "binance": "BNBUSDT",
        "bitmex": "BNBUSD",
        "apollox": "BNBUSDT",
        "drift": "BNB-PERP"
    },
]

benchmark_exchange = "binance"

result = []

for exchanges_market in exchanges_markets:
    market = exchanges_market[benchmark_exchange]
    hodl_df = hodl_df_map[market]

    row = {}
    row['symbol'] = market
    row['exchange'] = f'{benchmark_exchange} hodl'
    row['start_date'] = hodl_df.iloc[0]['datetime']
    row['end_date'] = hodl_df.iloc[-1]['datetime']
    row['num_date'] = (row['end_date'] - row['start_date']).days

    row['pnl'] = hodl_df.iloc[-1]['pnl'] * 100
    row['max_drawdown'] = max_drawdown(hodl_df['pnl']) * 100
    row['corr'] = hodl_df['pnl'].corr(hodl_df['pnl'])

    row['annualized_pnl'] = row['pnl'] / row['num_date'] * 365
    result.append(row)

    for exchange, market in exchanges_market.items():

        data_df = results[market][exchange]
        
        row = {}
        row['symbol'] = market
        row['exchange'] = exchange
        row['start_date'] = data_df.iloc[0]['datetime']
        row['end_date'] = data_df.iloc[-1]['datetime']
        row['num_date'] = (row['end_date'] - row['start_date']).days

        row['pnl'] = data_df.iloc[-1]['final_pnl'] * 100
        row['max_drawdown'] = max_drawdown(data_df['final_pnl']) * 100
        row['corr'] = data_df['final_pnl'].corr(hodl_df['pnl'])

        row['annualized_pnl'] = row['pnl'] / row['num_date'] * 365

        result.append(row)

result_df = pd.DataFrame(result)
result_df.to_csv("./output/basic_funding.csv")

print(result_df)

In [ ]:
# Leveraged funding arbitrage

# Fetch and save funding rate history PNL 
from datetime import datetime, timedelta

# Input
min_time = datetime.now() - timedelta(days=365 * 2)
max_time = datetime.now()
exchange = 'bitmex'
market = 'XRPUSD'

# Initial variables
funding_df_map = {}
price_df_map = {}
results = {}

max_leverage = 10

# Fetch data
funding_df = fetcher.fetch_funding_rate_history_until_start(exchange, market)
funding_df['datetime'] = funding_df['datetime'].dt.tz_localize(None)
funding_df = funding_df[(funding_df['datetime'] >= min_time) & (funding_df['datetime'] <= max_time)]

price_df = fetcher.fetch_ohlc(exchange, market, funding_df['timestamp'].min(), funding_df['timestamp'].max())
price_df['datetime'] = price_df['datetime'].dt.tz_localize(None)

funding_df = funding_df[(funding_df['datetime'] >= price_df['datetime'].min()) & (funding_df['datetime'] <= price_df['datetime'].max())]

input_df = pd.merge_asof(funding_df, price_df, on=['datetime'], tolerance=pd.Timedelta('1h'), direction='nearest')

input_df['open'] = input_df['open'].fillna(method='ffill')
input_df['high'] = input_df['high'].fillna(method='ffill')
input_df['low'] = input_df['low'].fillna(method='ffill')
input_df['close'] = input_df['close'].fillna(method='ffill')

# Fetch price data and calculate backtest result and save result to file
for leverage in range(1, max_leverage + 1):
    result_df = get_backtest_result(input_df, leverage)
    results[leverage] = result_df
    result_df.to_csv(f'./data/{exchange}_{market}_{leverage}x.csv', index=False)

# Hodl result
hodl_df = get_hodl_result(input_df)
hodl_df['datetime'] = hodl_df['datetime'].dt.tz_localize(None)
hodl_df.to_csv(f'./data/{exchange}_{market}_hodl.csv', index=False)

In [ ]:
# Plot leveraged funding arbitrage result

# Plot setup
fig, ax = plt.subplots(figsize=(20, 12))
# Plot
for leverage in range(1, max_leverage + 1):
    result_df = results[leverage]
    ax.plot(result_df['datetime'], result_df['final_pnl'] * 100, label=f'{leverage}x PNL (%)')

ax.plot(hodl_df['datetime'], hodl_df['pnl'] * 100, label=f'HODL PNL (%)')

# Chart settings
lines, labels = ax.get_legend_handles_labels()
ax.set_title(f"Leveraged funding arbitrage {exchange} {market}")
ax.legend(lines, labels, loc='upper right', bbox_to_anchor=(0.125, 0.3))

plt.tight_layout()
plt.savefig(f'./images/leveraged_{exchange}_{market}.png', dpi=300, format='png')
plt.show()

In [ ]:
# Plot histogram showing which leverage level is the best
from matplotlib.ticker import MultipleLocator

# Inputs
exchanges_markets = [
    {
        "binance": "BTCUSDT",
        "bitmex": "XBTUSD",
        "apollox": "BTCUSDT",
        "drift": "BTC-PERP"
    },
    {
        "binance": "ETHUSDT",
        "bitmex": "ETHUSD",
        "apollox": "ETHUSDT",
        "drift": "ETH-PERP"
    },
    {
        "binance": "SOLUSDT",
        "bitmex": "SOLUSD",
        "apollox": "SOLUSDT",
        "drift": "SOL-PERP"
    },
    {
        "binance": "XRPUSDT",
        "bitmex": "XRPUSD",
        "apollox": "XRPUSDT",
        "drift": "XRP-PERP"
    },
    {
        "binance": "BNBUSDT",
        "bitmex": "BNBUSD",
        "apollox": "BNBUSDT",
        "drift": "BNB-PERP"
    },
]

best_leverages = []


# Calculation
for exchanges_market in exchanges_markets:
    for exchange, market in exchanges_market.items():
        best_leverage = 1
        max_pnl = 0
        for leverage in range(1, max_leverage + 1):
            result_df = pd.read_csv(f'./data/{exchange}_{market}_{leverage}x.csv')
            pnl = result_df.iloc[-1]['final_pnl']
            if pnl > max_pnl:
                max_pnl = pnl
                best_leverage = leverage

        num_date = len(pd.date_range(result_df['datetime'].min(), result_df['datetime'].max(), freq='D'))  
        apy = 365 / num_date * max_pnl
        best_leverages.append({
            "exchange": exchange,
            "market": market,
            "leverage": best_leverage,
            "num_date": num_date,
            "pnl": max_pnl,
            "apy": apy
        })

df = pd.DataFrame(best_leverages)
df.to_csv("./output/best_leverages.csv")

# Plot setup
fig, ax = plt.subplots(figsize=(20, 12))

plt.hist(df['leverage'], color='blue', edgecolor='black')    

# Chart settings
ax.set_title(f"Best leverage level")
ax.xaxis.set_major_locator(MultipleLocator(1))

plt.tight_layout()
# plt.savefig(f'./images/leveraged_{exchange}_{market}.png', dpi=300, format='png')
plt.show()